In [26]:
from iSparrow import PreprocessorBase
from iSparrow import ModelBase
from iSparrow import SparrowRecording
import tests.set_up_sparrow_env as sp
from pathlib import Path
from watchdog.events import LoggingEventHandler, FileSystemEventHandler
from watchdog.observers import Observer
import logging
import time
import sys

sys.path.append("..")

In [28]:
class AnalysisEventHandler(FileSystemEventHandler):
    def __init__(self, callback: callable):
        self.callback = callback

    def on_created(self, event):
        print(event.event_type, event.src_path)
        if Path(event.src_path).is_file() and Path(event.src_path).suffix == ".wav":
            self.callback(event.src_path)

In [25]:
# FIXME: add species predictor
class Runner:

    def load_model(self):
        self.preprocessor_module = utils.load_module(
            "ppm", self.model_dir / Path(self.model_name) / "preprocessor.py"
        )

        self.model_module = utils.load_module(
            "md", self.model_dir / Path(self.model_name) / "model.py"
        )

    def __init__(
        self,
        indir: str,
        outdir: str,
        model_name: str,
        preprocessor_config: dict = {},
        model_config: dict = {},
        recording_config: dict = {},
    ):
        observers = Observer()
        event_handler = AnalysisEventHandler()
        self.input = Path(indir)
        self.output = Path(outdir)
        self.config = config
        self.model_dir = Path(model_dir)
        self.model_name = model_name
        self.proprocessor_module = None
        self.model_module = None
        self.model = None
        self.preprocessor = None

        self.load_model(model_name)

        self.preprocessor = self.preprocessor_module.Preprocessor(**preprocessor_config)

        self.model = self.model_module.Model(
            self.model_dir / model_name, **model_config
        )

        self.recording = SparrowRecording(
            self.preprocessor, self.model, "", **recording_config
        )

        self.results = []

    def change_model(self, model_name):

        raise RuntimeError("Not yet supported, only untested skeleton code exists")

        self.model_name = model_name

        self.load_model()

        self.preprocessor = self.preprocessor_module.Preprocessor(**preprocessor_config)

        self.model = self.model_module.Model(
            self.model_dir / model_name, **model_config
        )

        self.recording.set_analyzer(model, preprocessor)

    def analyze(self, filename: str):
        self.recording.path = filename

        self.recording.analyze()

        self.result.append(recording.detections)

    def run(self):
        observer = Observer()
        event_handler = AnalysisEventHandler(self.analyze)
        observer.schedule(event_handler, self.input, recursive=True)

        observer.start()

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            observer.stop()
        except Exception:
            print("Something went wrong")
            observer.stop()
        observer.join()

In [ ]:
preprocessor_cfg = {
    sample_rate: 48000,
    overlap: 0.0,
    sample_secs: 3.0,
    resample_type: "kaiser_fast",
}

model_cfg = {
    num_threads: int = 1,
    sigmoid_sensitivity: float = 1.0,
    species_list_file: str = None,
}

recording_cfg = {
    date: datetime = None,
    lat: float = None,
    lon: float = None,
    species_presence_threshold: float = 0.03,
    min_conf: float = 0.25,
    species_predictor: SpeciesPredictorBase = None,
}